# ☕ Data Extraction: Merida Coffee Shops

This data acquisition part uses two services from the Google Places API within the Google Maps Platform to collect data for coffee shops in Merida, Yucatan.  

---

## Google Places API Workflow

This project follows a two-step extraction process:

1. **Places Text Search (New)** – performs text-based queries to extract the Place IDs.
2. **Place Details** – uses the collected Place IDs to retrieve the complete metadata for each place.

---

## 1. Defining the Search Area

The Google Places API provides two parameters to geographically constrain results: **locationBias** and **locationRestriction**.  
Since this project requires strict adherence to Merida’s municipal limits, the **locationRestriction** parameter is selected. It defines a bounding box that limits all returned results to the specified area, ensuring spatial precision and consistency.

#### Gridding the City with Folium

Because the Google Places API limits each query to 20 results, Merida’s area is subdivided into smaller rectangular viewports.  
Each grid cell defines a separate locationRestriction (SW and NE coordinates) and is queried independently with the Text Search service.  
Once all Place IDs are collected, the **Place Details** service is called to extract complete metadata for each identified place.

The **Folium** library is used to visualize and verify the grid layout, ensuring full coverage of the study area and validating the data extraction process.


In [29]:
import folium

#Create the map of Merida using an approximate center point.
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12)
delta = 0.01 #The variable “delta” controls how “large” each rectangle is.

initial_lat, initial_lng = 20.891532412575916, -89.73272017481521 #This is the initial SW point where the loop start to create the other viewport rectangles. 


#The points generated will be stored and will be used when the API is called.
rectangles_viewports = []

#Generate each SW and NE point from each rectangle using the initial point
for i in range(20):
    for j in range(20):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

In [ ]:
#Number of calls that will be made to the API
print(len(rectangles_viewports))

## 2. Data Acquisition
Since two Google Place API services will be used, we first need to extract the IDs of the places using the SearchText service, and then use the Place Details service to extract all the relevant information for each place.

#### Place IDs extraction


In [3]:
import time
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
#Loading the API key to make the requests
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [ ]:
#Define the API endpoint and headers to make the calls
search_url = "https://places.googleapis.com/v1/places:searchText"

search_headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id' #Since we will only extract the Place IDs in this part, only this parameter will be specified for it to be returned.
}

In [ ]:
raw_data = [] #The raw data returned by the API will be stored here

#Extract the Place IDs by iterating through each of the 255 rectangle viewports, which will be saved in a .json file.
for low, high in rectangles_viewports:
    try: 
        payload = {
            'textQuery' : 'cafeteria',
            'includedType': 'cafe',
            'strictTypeFiltering':True,
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = search_url, json=payload, headers = search_headers)
        time.sleep(0.5)
        response.raise_for_status()
        
        data = response.json()
        
        raw_data.extend(data.get('places', []))
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')
    

with open('data/places_ids.json', 'w') as f:
    json.dump(raw_data, f, indent=4)

In [4]:
#All place IDs are stored in a JSON file for use when using "Place Details" calls
with open('data/places_ids.json', 'r') as file:
    ids_dict = json.load(file)

In [ ]:
#Number of coffee shops found
len(ids_dict)

648

#### Places details data extraction


In [ ]:
details_headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask':'displayName,formattedAddress,postalAddress,location,businessStatus,primaryTypeDisplayName,priceRange,rating,userRatingCount,postalAddress,websiteUri'
}

In [ ]:
detail_raw_data = []

for place in ids_dict:
    try:
        details_url = f'https://places.googleapis.com/v1/places/{place['id']}'
        
        response = requests.get(url=details_url, headers=details_headers)
        time.sleep(0.5)
        response.raise_for_status()
        
        data = response.json()
        
        detail_raw_data.append(data)
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')

pd.DataFrame(detail_raw_data).to_csv('data/raw_df.csv')

In [8]:
pd.read_csv('data/raw_df.csv', index_col=0).head()

,id,formattedAddress,location,rating,businessStatus,userRatingCount,displayName,primaryTypeDisplayName,postalAddress,priceRange,websiteUri
0,ChIJ21xPrRcTVo8R8NtbxJTwrAY,"C. 27 117, San Lorenzo, Umán, 97390 Umán, Yuc....","{'latitude': 20.894740499999997, 'longitude': ...",5.0,OPERATIONAL,2.0,"{'text': 'Frappe shop suc san lorenzo', 'langu...","{'text': 'Cafe', 'languageCode': 'en-US'}","{'regionCode': 'MX', 'languageCode': 'en-US', ...",NaN,NaN
1,ChIJiy4UiFITVo8R2gZm2huVwGI,"C. 27 131, San Lorenzo, Umán, 97390 Umán, Yuc....","{'latitude': 20.8945503, 'longitude': -89.732288}",3.0,OPERATIONAL,5.0,"{'text': 'Panucheria Doña Yolí', 'languageCode...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}","{'regionCode': 'MX', 'languageCode': 'en-US', ...",NaN,NaN
2,ChIJ_2E3cAATVo8RG3CRsaAiel4,"C. 20, Umán, 97390 Umán, Yuc., Mexico","{'latitude': 20.8915676, 'longitude': -89.7302...",5.0,OPERATIONAL,1.0,"{'text': 'Taquería mi Angel', 'languageCode': ...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}","{'regionCode': 'MX', 'languageCode': 'en-US', ...",NaN,NaN
3,ChIJndc0PwBtVo8RbDj1kFhr-4E,"Campeche-Mérida 11592, 97392 Mérida, Yuc., Mexico","{'latitude': 20.8941657, 'longitude': -89.6934...",NaN,OPERATIONAL,NaN,"{'text': 'Barra andatti Gourmet Santa Cruz', '...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}","{'regionCode': 'MX', 'languageCode': 'en-US', ...",NaN,NaN
4,ChIJZ4kqBaVtVo8RYIOkor7j8ZY,"97295 Merida, Yucatan, Mexico","{'latitude': 20.898615600000003, 'longitude': ...",4.5,OPERATIONAL,14.0,"{'text': ""Moheno's Café"", 'languageCode': 'es'}","{'text': 'Cafe', 'languageCode': 'en-US'}","{'regionCode': 'MX', 'languageCode': 'en-US', ...",NaN,NaN
